In [1]:
import multiprocessing as mp
import test
import time

In [2]:
with mp.Manager() as manager:
    tmp_d = manager.list([False])

    p = mp.Process(target=test.check, args=(tmp_d,))
    p.start()

    time.sleep(2)
    tmp_d[0] = True
    p.join()

start process
haha


In [3]:
tmp_d

<DictProxy object, typeid 'dict' at 0x1101e1580>

In [4]:
p

<Process name='Process-2' pid=73520 parent=73316 stopped exitcode=1>